In [1]:
import pyspark
import pyspark.sql
from pyspark.sql import *
from pyspark.sql.functions import *
import json
import urllib
import argparse
from pyspark.ml.feature import CountVectorizerModel
from pyspark.ml.clustering import LDA
%matplotlib inline

conf = pyspark.SparkConf().setMaster("local[*]").setAll([
                                   ('spark.driver.memory','240g'),
                                   ('spark.driver.maxResultSize', '32G'),
                                   ('spark.local.dir', '/scratch/tmp/'),
                                   ('spark.yarn.stagingDir', '/scratch/tmp/'),
                                   ('spark.sql.warehouse.dir', '/scratch/tmp/')
                                  ])

# create the session
spark = SparkSession.builder.config(conf=conf).getOrCreate()
# create the context
sc = spark.sparkContext


In [2]:
spark

In [3]:
WIKIDATA_MAPPING = 'dumps/WikidataInterlanguageLinks.parquet'
sites = ["enwiki", "eswiki", "itwiki", "dewiki", "frwiki", "cswiki", 
         "cawiki", "svwiki", "arwiki", 
             "elwiki",  "fiwiki", 
             "hewiki", "idwiki",  "kowiki", "nlwiki", "plwiki", 
             "ptwiki", "rowiki", "ruwiki", "sqwiki", "srwiki", "trwiki", 
             "ukwiki", "viwiki", "warwiki", "zhwiki", "fawiki", "jawiki"]

wikidata = spark.read.parquet(WIKIDATA_MAPPING)
all_entities = sc.emptyRDD()
for i in range(len(sites)):
    site = sites[i]
    all_entities = all_entities\
            .union(wikidata.where("site='{}'".format(site)).selectExpr("qid", "title", str(i)+" as position").rdd)

wikidata_tmp = all_entities\
        .map(lambda r: (r.qid, [r])).reduceByKey(lambda a,b: a+b)\
        .map(lambda r: Row(qid=r[0], titles=[t.title for t in sorted(r[1], key=lambda x: x.position)])).collect()

wikidata_titles = {}
for w in wikidata_tmp:
    wikidata_titles[w.qid]=w.titles

del wikidata_tmp

In [7]:
import numpy
from pyspark.ml.clustering import LocalLDAModel

k = 35

model = LocalLDAModel.load("models/EnrichedLinks/LDA_model_{}.model".format(k))
topics_info = model.describeTopics(maxTermsPerTopic=15).collect()

vocab = numpy.array(transformer.vocabulary)
for t in topics_info:
    print(t.topic, [wikidata_titles[vocab[w]][0] for w in t.termIndices])


0 ['Israel', 'Jerusalem', 'United States', 'Jesus', 'Catholic Church', 'Christianity', 'Bible', 'Hebrew language', 'William Shakespeare', 'Judaism', 'Jews', 'Tel Aviv', 'New Testament', 'Islam', 'Eastern Orthodox Church']
1 ['Asteroid belt', 'Astronomical unit', 'Indonesia', 'Asteroid', 'Celsius', 'Jahresmitteltemperatur', 'Precipitation', 'Jahresniederschlag', 'Jakarta', 'Semieje mayor', 'Orbital eccentricity', 'Kilometre', 'Watercourse', 'Julian day', 'Ecliptic']
2 ['Turkey', 'Administrative divisions of Turkey', 'Istanbul', 'Eastern European Time', 'Eastern European Summer Time', 'Chile', 'Argentina', 'United States', 'Santiago', 'NASA', 'Uruguay', 'Montevideo', 'Italy', 'Country', 'Basketball']
3 ['Scuderia Ferrari', 'Ford Motor Company', 'Czech Republic', 'McLaren', 'Cosworth', 'Williams Grand Prix Engineering', 'Formula One', 'Renault F1', 'Prague', 'Mercedes-Benz', 'Central European Time', 'Institut national de la statistique et des études économiques', 'Michael Schumacher', 'Re

In [9]:
import numpy
from pyspark.ml.clustering import LocalLDAModel

k = 150

model = LocalLDAModel.load("models/EnrichedLinks/LDA_model_{}.model".format(k))
topics_info = model.describeTopics(maxTermsPerTopic=15).collect()

vocab = numpy.array(transformer.vocabulary)
for t in topics_info:
    print(t.topic, [wikidata_titles[vocab[w]][0] for w in t.termIndices])


Py4JJavaError: An error occurred while calling o726.load.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/dlabdata1/piccardi/WikiPDA/models/EnrichedLinks/LDA_model_150.model/metadata
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:287)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:204)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:273)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:269)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:269)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:273)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.app

In [8]:
import numpy
from pyspark.ml.clustering import LocalLDAModel

k = 210

model = LocalLDAModel.load("models/EnrichedLinks/LDA_model_{}.model".format(k))
topics_info = model.describeTopics(maxTermsPerTopic=15).collect()

vocab = numpy.array(transformer.vocabulary)
for t in topics_info:
    print(t.topic, [wikidata_titles[vocab[w]][0] for w in t.termIndices])


0 ['Jesus', 'Christianity', 'Bible', 'Catholic Church', 'New Testament', 'Old Testament', 'Martin Luther', 'Eastern Orthodox Church', 'Protestantism', 'Paul the Apostle', 'Eucharist', 'Baptism', 'Gospel of Matthew', 'Mary, mother of Jesus', 'Trinity']
1 ['Asteroid belt', 'Astronomical unit', 'Asteroid', 'Semieje mayor', 'Orbital eccentricity', 'Kilometre', 'Julian day', 'Ecliptic', 'Julian year (astronomy)', 'Minor Planet Center', 'Orbital inclination', 'Degree (angle)', 'Lincoln Near-Earth Asteroid Research', 'Jet Propulsion Laboratory', 'Absolute magnitude']
2 ['Administrative divisions of Turkey', 'Eastern European Summer Time', 'Turkey', 'Eastern European Time', 'Country', 'International Space Station', 'Socialist Party (Portugal)', 'Social Democratic Party (Portugal)', 'Coventry Climax', "CDS – People's Party", 'Unitary Democratic Coalition', 'NASA', 'Astronaut', 'Red Star Belgrade', 'Space Shuttle']
3 ['Coral', 'Mugen Motorsports', 'Cnidaria', 'Benetton Formula', 'Ford Motor Comp